In [1]:
from topic_modeling_functions import *

pd.set_option('display.max_colwidth', -1)

In [2]:
tfidf_vect = TfidfVectorizer(lowercase=False, tokenizer=lambda r: r)

Загрузим данные:

In [3]:
df_countries = pd.read_pickle('countries_topic_modeling.pkl')

Посчитаем количество:

In [4]:
df_countries['country'].count()

45

In [5]:
df_countries['country']

0     18 век         
1     19 век         
2     Австралийская  
3     Австрийская    
4     Аргентинская   
5     Армянская      
6     Белорусская    
7     Бельгийская    
8     Бразильская    
9     Британская     
10    Гватемальская  
11    Грузинская     
12    Датская        
13    Древнегреческая
14    Древнеримская  
15    Древность      
16    Еврейская      
17    Индийская      
18    Ирландская     
19    Исландская     
20    Испанская      
21    Итальянская    
22    Китайская      
23    Колумбийская   
24    Кубинская      
25    Немецкая       
26    Нидерландская  
27    Норвежская     
28    Персидская     
29    Польская       
30    Португальская  
31    США            
32    Советская      
33    Современная    
34    Турецкая       
35    Украинская     
36    Финская        
37    Французская    
38    Чешская        
39    Чилийская      
40    Шведская       
41    Швейцарская    
42    Шотландская    
43    Югославская    
44    Японская       
Name: coun

Выберем литературу, с которой будем работать (в переменную ```index``` запишем индекс из списка выше, соответствующий нужной литературе):

In [6]:
index = 0
df_selected = df_countries.iloc[index]

Преобразуем тексты в вектора с помощью TfidfVectorizer:  
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [7]:
tfidf_data, tfidf_feature_names = get_data_vectorized(tfidf_vect, df_selected.text_pymystem_list)

Выберем интервал для количества топиков, которые хотим получить:  
* ```start``` - минимальное количество топиков
* ```stop``` - максимальное количество топиков + 1
* ```step``` - шаг (например, если при заданных значениях ```start``` и ```stop``` мы хотим посмотреть результаты для количества топиков 5, 10, 15, 20, то выбираем ```step``` = 5)

In [8]:
start = 5  # min n_topics
stop = 21  # max n_topics + 1
step = 1

Создадим и обучим модели для выбранного интервала значений:

In [9]:
nmf_res = get_models_for_n_topics('nmf', tfidf_data, start, stop, step)

Зададим количество слов для каждого топика, которые хотим получить:

In [10]:
n_top_words = 10

Получим нужное количество слов для каждого топика:

In [11]:
nmf_topics = get_n_top_words(nmf_res, tfidf_feature_names, n_top_words)

Создадим датафрейм из полученных результатов:

In [12]:
df_results = pd.Series(nmf_topics, index=get_index(start, min(tfidf_data.shape[0] + 1, stop), step))

Выберем нужное количество топиков из интервала, который мы задавали ранее:  
(в ```n_topics_5``` заменим ```5``` на нужное число)

In [13]:
df_results.loc['n_topics_5']

n_topic_1    [герой, корабль, замок, молодой, остров, борнгольм, человек, старец, ветер, ребенок]            
n_topic_2    [рурик, вадим, рамида, пренеста, новгород, пренест, извед, вигор, народ, венец]                 
n_topic_3    [анюта, мельник, виктор, филат, фетинья, филимона, филимон, анкудин, мирон, крестьянин]         
n_topic_4    [верхолет, полист, простодум, милена, честон, марина, чванкина, чванкин, замира, замир]         
n_topic_5    [иванушка, советник, советница, бригадир, софья, бригадирша, добролюбов, иван, авдотья, акулина]
dtype: object

Выберем номер топика для нужного количества топиков (в ```n_topics_5``` заменим ```5``` на нужное количество топиков и в ```n_topic_1``` заменим ```1``` на нужный номер топика):

In [14]:
df_results.loc[('n_topics_5', 'n_topic_1')]

['герой',
 'корабль',
 'замок',
 'молодой',
 'остров',
 'борнгольм',
 'человек',
 'старец',
 'ветер',
 'ребенок']

Сохраним полученные результаты в файл .tsv (названием файла служит название литературы, а разделителем -- табуляция ```\t```):

In [15]:
df_results.to_csv(df_selected['country'] + '.tsv', sep='\t')

Если хотим транслитерировать название файла, то скачиваем и импортируем модуль ```transliterate``` https://pypi.org/project/transliterate/, а затем применяем функцию ```translit``` к названию литературы (заменяя пробелы на нижние подчеркивания ```_``` и убирая ```'```):

In [16]:
from transliterate import translit

In [17]:
transliterated = translit(df_selected['country'].replace(' ', '_'), 'ru', reversed=True).replace("'", '')
df_results.to_csv(transliterated + '.tsv', sep='\t')